## Get data from Meetup.com

We are going to get data from Meetup's API. You can create custom requests using their API console, then make them via Python by copying the secure link here.

https://secure.meetup.com/meetup_api/console/?path=/2/members

In [4]:
import pandas as pd
import numpy as np
import requests
import os
import json
from time import sleep

#### Get PyNash Members

In [46]:
        tdf = pd.DataFrame.from_dict(response['results']).set_index('id')


KeyError: 'id'

In [50]:
tdf

,city,hometown,joined,lat,lon,name,state,visited
id,,,,,,,,
199688329,Nashville,NaN,1499500083000,36.08,-86.72,Caroline S,TN,1501927876000
16881781,Brentwood,NaN,1488293512000,36.00,-86.79,Debbie Deaver,TN,1491332655000
19366751,Franklin,NaN,1490552583000,35.92,-86.89,Debbie R,TN,1497275201000
222900319,Fairview,NaN,1488303307000,35.97,-87.13,Dee,TN,1488303307000
222879779,Fairview,NaN,1488273913000,35.97,-87.13,Donna Beasley Rayburn,TN,1488273913000
77351042,Nolensville,NaN,1488378187000,35.94,-86.67,John Lavender,TN,1488378187000
222774631,Fairview,Fairview,1488130648000,35.97,-87.13,Jon Cherry,TN,1490659763000
197426336,Fairview,NaN,1488248903000,35.97,-87.13,Matt and Kelley,TN,1488310106000
170786152,Fairview,NaN,1488248861000,35.97,-87.13,Matthew Bush,TN,1488248861000


In [66]:
group_id = 22504351
def get_group_members(group_id):

    # Initialize variables
    members = pd.DataFrame()
    page = 0
    bad_iters = 0
    
    # Keep querying until there are no more results
    all_results = False
    while all_results == False:
        q = 'https://api.meetup.com/2/members?&sign=true&photo-host=public&group_id={}&only=name,id,city,state,hometown,joined,visited,lat,lon&page=200&offset={}&key=1eb16676d664fa48314391ae5b6c'.format(group_id, page)
        response = requests.get(q).json()
        if 'results' in response.keys():
            try:
                tdf = pd.DataFrame.from_dict(response['results']).set_index('id')
                members = members.append(tdf)
                page += 1
            except KeyError as exc:
                all_results = True
                bad_iters += 1
                if bad_iters > 10:
                    return None
                pass
        else:
            sleep(2)
            pass
        
    return members

In [26]:
def agg_group_members(list_of_group_ids, write_path=None):
    all_members = pd.DataFrame()
    
    for g in list_of_group_ids:
        sleep(1)
        try:
            tdf = get_group_members(g)
            all_members = all_members.append(tdf)
        except KeyError as exc:
            print(group, exc)
            continue

    # Write to computer
    if write_path:
        all_members.to_csv(write_path, encoding='utf-8') 
    
    return all_members

In [19]:
#### from time import sleep
import json

def get_all_groups(location_str, radius=25, write_path=None):
    cols = ('group_id', 'group_name', 'num_members', 'category_id', 
            'category_name', 'organizer_id', 'group_urlname')
    all_groups = pd.DataFrame(columns=cols)

    for page in np.arange(10):
        q = 'https://api.meetup.com/find/groups?&sign=true&photo-host=public&location={}&radius={}&page=200&offset={}&key=1eb16676d664fa48314391ae5b6c'.format(location_str, radius, page) 
        response = requests.get(q).json()
        if len(response) > 0:
            try:
                for g in response:
                    s = pd.Series((g['id'], g['name'], g['members'], g['category']['id'], 
                                   g['category']['name'], g['organizer']['id'], g['urlname']),
                                 index=cols)
                    all_groups = nash_groups.append(s, ignore_index=True)
            except: 
                continue
        # Sleep briefly so that API doesn't get overwhelmed
        sleep(0.2)

    all_groups = all_groups.astype({'group_id': int, 'organizer_id': int, 'category_id': int, 'num_members': int})
    all_groups.set_index('group_id', inplace=True)
    
    # Write to computer
    if write_path:
        all_groups.to_csv(write_path) 

    return all_groups

groups = get_all_groups('nashville tn', 'data_groups.csv')

In [48]:
groups.tail()

,group_name,num_members,category_id,category_name,organizer_id,group_urlname
group_id,,,,,,
22504351,Sumner County Shooters,32,32,Sports & Recreation,6141229,SumnerCountyShooters
21686664,Gallatin Makerspace,118,34,Tech,126066112,GallatinMakerspace
22644046,First District Conservatives,15,13,Movements & Politics,222774631,First-District-Conservatives
19494897,The Relationship Sounding Board,40,33,Support,186028004,The-Relationship-Sounding-Board
19494897,The Relationship Sounding Board,40,33,Support,186028004,The-Relationship-Sounding-Board


#### Get memberships for each member

In [27]:
from time import sleep
import json


edges = pd.DataFrame()
err_ids = []
for pid in members.index:
    r = requests.get('https://api.meetup.com/2/groups?&sign=true&member_id={}&page=200&key=1eb16676d664fa48314391ae5b6c'.format(pid))
    try:
        r = r.json()
        for membership in r['results']:
            edge = pd.Series({'member_id': pid, 
                              'group_id': membership['id'], 
                              'group_name': membership['name']})
            edges = edges.append(edge, ignore_index=True)
    except json.decoder.JSONDecodeError:
        print(pid)
        err_ids.append(pid)
    
    # Sleep briefly so that API doesn't get overwhelmed
    sleep(0.2)
        

# Write to computer
write_data = True
if write_data == True:
    edges.to_csv('data_edges.csv') 
